In [ ]:
# GPU and auto-reload
# (In the Colab UI: Runtime ▷ Change runtime type ▷ GPU)
%load_ext autoreload
%autoreload 2


In [ ]:
    # In your local_test_runner.ipynb (or modified colab_runner.ipynb) - Cell 1
    from google.colab import drive # This line will do nothing locally, or you can comment it out
    # drive.mount('/content/drive') # Comment this out for local execution

    # --- FOR LOCAL TESTING ---
    import os
    # Define a base directory for your local "drive"
    LOCAL_PROJECT_BASE = './temp_colab_env_data' # Or an absolute path like '/Users/yourname/my_project_data'
    os.makedirs(LOCAL_PROJECT_BASE, exist_ok=True) # Create it if it doesn't exist

    DRIVE_ROOT = os.path.abspath(LOCAL_PROJECT_BASE) # Use absolute path for clarity
    # --- END LOCAL TESTING SETUP ---

    # The rest of Cell 1 remains the same, using the new DRIVE_ROOT
    DATA_ROOT  = f'{DRIVE_ROOT}/datasets/meld_data'
    CKPT_ROOT  = f'{DRIVE_ROOT}/checkpoints'
    HF_CACHE_ROOT = f'{DRIVE_ROOT}/hf_cache'

    os.makedirs(DATA_ROOT, exist_ok=True)
    os.makedirs(CKPT_ROOT, exist_ok=True)
    os.makedirs(HF_CACHE_ROOT, exist_ok=True)
    os.makedirs(f"{HF_CACHE_ROOT}/datasets", exist_ok=True)
    os.makedirs(f"{HF_CACHE_ROOT}/models", exist_ok=True)


    print(f"Local testing mode. Project data will use base: {DRIVE_ROOT}")
    # ... (rest of the print statements)

In [ ]:
#Clone or Pull Repo
import pathlib

REPO_URL = "https://github.com/mehulbhardwaj/emotion-classifier-capstone.git"
REPO_DIR = "/content/emotion-classifier-capstone"

if pathlib.Path(REPO_DIR).exists():
    %cd $REPO_DIR
    !git pull --quiet
else:
    !git clone $REPO_URL $REPO_DIR --quiet
    %cd $REPO_DIR


In [ ]:
#Cell 3 – Install dependencies via pip
!pip install -U pip wheel
!pip install -q -r requirements.txt

# Cache HF models & datasets on Drive
import os
os.environ["HF_HOME"]        = f"{DRIVE_ROOT}/hf_cache"
os.environ["HF_DATASETS_CACHE"] = f"{DRIVE_ROOT}/hf_cache"


In [ ]:
#Cell 4 – Emit a session-specific YAML override
import pathlib, textwrap

cfg_text = textwrap.dedent(f"""
defaults:
  - _self_
  - model: mlp_fusion
  - trainer: colab_gpu

paths:
  data_root: {DATA_ROOT}
  ckpt_root: {CKPT_ROOT}

training:
  seed: 42
""")

path_cfg = pathlib.Path("configs/path_colab.yaml")
path_cfg.write_text(cfg_text)
print(path_cfg.read_text())


In [ ]:
#Cell 5 – Run (or resume) training
# ────────────────────────────────────────────────────────────────────────────
# This picks up `last.ckpt` (if present), writes new ckpts every 500 steps,
# and now cfg.experiment_name is guaranteed to exist.
# ────────────────────────────────────────────────────────────────────────────
!python main.py \
   --config_file configs/path_colab.yaml \
   --architecture mlp_fusion \
   --train_model \
   --num_epochs 1 \
   --experiment_name "mlp_fusion_local_smoke_test" \
   --limit_dialogues_train 50 \
   --limit_dialogues_dev 10 \
   --limit_dialogues_test 10 \
   --batch_size 16 \
   --learning_rate 3e-4 \
   --experiment_name mlp_fusion


In [ ]:
#Cell 6 – Validation / Inference
!python main.py \
   --config_file configs/path_colab.yaml \
   --evaluate_model {CKPT_ROOT}/mlp_fusion/last.ckpt
